# Redis列表实现一次pop 弹出多条数据

![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


In [24]:
# 连接 Redis

import redis
client = redis.Redis(host='122.51.39.219', port=6379, password='leftright123')

# 注意：
# 这个 Redis 环境仅作为练习之用，每小时会清空一次，请勿存放重要数据。

In [25]:
# 准备数据

client.lpush('test_batch_pop', *list(range(10000)))

10000

In [6]:
# 一条一条读取，非常耗时
import time


start = time.time()
while True:
    data = client.lpop('test_batch_pop')
    if not data:
        break
end = time.time()

delta = end - start
print(f'循环读取10000条数据，使用 lpop 耗时：{delta}')

循环读取10000条数据，使用 lpop 耗时：112.04084920883179


## 为什么使用`lpop`读取10000条数据这么慢？

因为`lpop`每次只弹出1条数据，每次弹出数据都要连接 Redis 。大量时间浪费在了网络传输上面。

## 如何实现批量弹出多条数据，并在同一次网络请求中返回？

先使用 `lrange` 获取数据，再使用`ltrim`删除被获取的数据。

In [26]:
# 复习一下 lrange 的用法

datas = client.lrange('test_batch_pop', 0, 9)  # 读取前10条数据
datas

[b'9999',
 b'9998',
 b'9997',
 b'9996',
 b'9995',
 b'9994',
 b'9993',
 b'9992',
 b'9991',
 b'9990']

In [28]:
# 学习一下 ltrim 的用法

client.ltrim('test_batch_pop', 10, -1)  # 删除前10条数据

True

In [29]:
# 验证一下数据是否被成功删除

length = client.llen('test_batch_pop')
print(f'现在列表里面还剩{length}条数据')
datas = client.lrange('test_batch_pop', 0, 9)  # 读取前10条数据
datas

现在列表里面还剩9990条数据


[b'9989',
 b'9988',
 b'9987',
 b'9986',
 b'9985',
 b'9984',
 b'9983',
 b'9982',
 b'9981',
 b'9980']

In [30]:
# 一种看起来正确的做法

def batch_pop_fake(key, n):
    datas = client.lrange(key, 0, n - 1)
    client.ltrim(key, n, -1)
    return datas

batch_pop_fake('test_batch_pop', 10)

[b'9989',
 b'9988',
 b'9987',
 b'9986',
 b'9985',
 b'9984',
 b'9983',
 b'9982',
 b'9981',
 b'9980']

In [31]:
client.lrange('test_batch_pop', 0, 9)

[b'9979',
 b'9978',
 b'9977',
 b'9976',
 b'9975',
 b'9974',
 b'9973',
 b'9972',
 b'9971',
 b'9970']

## 这种写法用什么问题

在多个进程同时使用 batch_pop_fake 函数的时候，由于执行 lrange 与 ltrim 是在两条语句中，因此实际上会分成2个网络请求。那么当 A 进程
刚刚执行完lrange，还没有来得及执行 ltrim 时，B 进程刚好过来执行 lrange，那么 AB 两个进程就会获得相同的数据。

等 B 进程获取完成数据以后，A 进程的 ltrim 刚刚抵达，此时Redis 会删除前 n 条数据，然后 B 进程的 ltrim 也到了，再删除前 n 条数据。那么最终导致的结果就是，AB 两个进程同时拿到前 n 条数据，但是却有2n 条数据被删除。

## 使用 pipeline 打包多个命令到一个请求中

pipeline 的使用方法如下：

```python
import redis

client = redis.Redis()
pipe = client.pipeline()
pipe.lrange('key', 0, n - 1)
pipe.ltrim('key', n, -1)
result = pipe.execute()
```

pipe.execute()返回一个列表，这个列表每一项按顺序对应每一个命令的执行结果。在上面的例子中，result 是一个有两项的列表，第一项对应 lrange 的返回结果，第二项为 True，表示 ltrim 执行成功。

In [32]:
# 真正可用的批量弹出数据函数

def batch_pop_real(key, n):
    pipe = client.pipeline()
    pipe.lrange(key, 0, n - 1)
    pipe.ltrim(key, n, -1)
    result = pipe.execute()
    return result[0]

In [33]:
# 清空列表并重新添加10000条数据
client.delete('test_batch_pop')
client.lpush('test_batch_pop', *list(range(10000)))

10000

In [34]:
start = time.time()
while True:
    datas = batch_pop_real('test_batch_pop', 1000)
    if not datas:
        break
    for data in datas:
        pass
end = time.time()
print(f'批量弹出10000条数据，耗时：{end - start}')

批量弹出10000条数据，耗时：0.18534111976623535


In [35]:
client.llen('test_batch_pop')

0

![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/640.gif)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)